# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.enable('data_server')
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


## Ejercicio 1 y 3

(10 puntos)

`Mostrar cantidad de fallecidos la fecha por cada grupo etáreo:`


(10 puntos)

`Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.`

**Respuesta:**

Para esto usaremos `fallecidos_etareo`. Necesitamos entregar la cantidad de fallecidos por grupo etareo dada una fecha, que es más informativo que dado un grupo etareo entregar fallecidos en una fecha. Concretamente, buscamos un gráfico con eje X las fechas y el eje Y cantidad de fallecidos. Como tenemos una cantidad de grupos etareos de


In [7]:
fallecidos_etareo["grupo_de_edad"].drop_duplicates().shape

(7,)

podriamos asociar la información con un gráfico de lineas ya que 7 colores está dentro de un rango perceptible. Veamos como es el gráfico:

In [8]:
line = alt.Chart(fallecidos_etareo).mark_line().encode(
    #note que tenemos muchas fechas asique mostramos por meses
    x=alt.Y('fecha',axis=alt.Axis(title='fecha (semanas)')),
    y='fallecidos',
    color=alt.Color('grupo_de_edad', legend=alt.Legend(title="Grupo etareo")),
    tooltip=['grupo_de_edad','fecha','fallecidos']
).properties(
    title='Cantidad fallecidos por grupo etareo',
    width=700, 
    height=300
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none')

selectors = alt.Chart(fallecidos_etareo).mark_point().encode(
    x='fecha',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=-30, dy=-20,color='black').encode(
    text=alt.condition(nearest, 'fallecidos:Q', alt.value(''))
)

# Draw a rule at the location of the selection
rules = alt.Chart(fallecidos_etareo).mark_rule(color='gray').encode(
    x='fecha',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
)

alt.LayerChart(...)

**Comentarios:** 

Como se dijo, se condensa la información facilmente en este tipo de gráfico: seleccionamos grupo etareo y buscamos por fecha cuantos fallecidos o cómo fue su evolución en el tiempo. Notemos que todas las curvas parecen distribuirse de la misma forma solo que unas mas amortiguadas que otras, lo que nos podria conducir a que tienen un modelo subyacente igual con distinto parámetro. 

También nos podemos dar cuenta de que a mitad de julio, que es fecha posterior (algunos días de diferencia solamente) a cuando el ex-ministro Mañalich renuncia a su cargo ([video fecha](https://www.youtube.com/watch?v=0pSEqB_cozA)), hay una irregularidad en todas las curvas, entonces podemos suponer que la forma de conteo cambió con este suceso (o algún mecanismo que genera los datos). Incluso, después de este quiebre las curvas tienden a mantener su crecimiento o al menos es menor.

Por otro lado, si ordenamos de menor mayor crecimiento tenemos

| grupo etareo      | puesto     |
| :------------- | :----------: |
| <39 | 1|
| 40-49 | 2 |
| 50-59| 4|
|60-69 | 5|
| 70-79 | 7|
| 80-89 | 6|
| >90 | 3|

De otra forma, consideremos la media de cada curva y hagamos un histograma junto con uno de cantidad de fallecidos a la fecha. Esto tiene sentido ya que tienen una distribucion similar y son crecientes.

In [9]:
medias = fallecidos_etareo.groupby("grupo_de_edad").mean().reset_index()

pregunta3_anex = alt.Chart(medias).mark_bar().encode(
    x=alt.X('grupo_de_edad',axis=alt.Axis(title='grupo etareo')),
    y='fallecidos',
).properties(
    title='Cantidad promedio fallecidos por grupo etareo',
    width=300, 
    height=200
)

pregunta1 = alt.Chart(fallecidos_etareo.groupby("grupo_de_edad").sum().reset_index()).mark_bar().encode(
    x=alt.X('grupo_de_edad',axis=alt.Axis(title='grupo etareo')),
    y='fallecidos',
).properties(
    title='Cantidad fallecidos por grupo etareo a la fecha',
    width=300, 
    height=200
)

pregunta3_anex | pregunta1

alt.HConcatChart(...)

que es justamente lo que se dijo. Note además que el supuesto de las medias tiene sentido.

## Ejercicio 2

(10 puntos)

`¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.`

**Respuesta:**

Primero veamos que conjunto de datos entre `activos` y `confirmados` tiene más elementos, ya que estos dos poseen lo que necesitamos

In [10]:
activos.loc[:, ["comuna", "poblacion"]].drop_duplicates().shape[0]-confirmados.loc[:, ["comuna", "poblacion"]].drop_duplicates().shape[0]

1

Entonces elegimos `activos`(no es significativa esta elección, incluso posiblemente es una columna que no corresponda a una comuna). Notemos que tenemos 

In [11]:
activos.loc[:, ["comuna", "poblacion"]].drop_duplicates().shape[0]

346

como cantidad de comunas (sabemos que hay 346 comunas en chile asique tenemos todas). Con esto, como no tenemos un orden sobre las comunas, tan solo nos queda cuantificar cuan grande es con otra en comparacion de su población, por lo que haremos un gráfico de área ordenando de mayor a menor. Esto tiene sentido ya que, por ejemplo, con el orden natural que entrega altair (alfabéticamente), perdemos la información de facil acceso a las comunas de mayor (menor) poblacion, que es, como dijimos, la información de interes (y la única que podemos extraer). Además, se considera un reescalamiento del eje de la población (eje Y) a uno logarítmico, dado que la razon $\min_i(poblacion_i) /\max_i(poblacion_i)$ es muy pequeña y sería poco informativo un gráfico con escala normal. Exactamente esta razón es

In [12]:
activos.loc[:, ["comuna", "poblacion"]].drop_duplicates()["poblacion"].min()/activos.loc[:, ["comuna", "poblacion"]].drop_duplicates()["poblacion"].max()*100

0.021210418185843516

En concreto, podremos saber,cualitativamente, cuan grande es el valor máximo, pero sería difícil detectar cuan pequeño es el mínimo. Veamos cómo es el gráfico

In [13]:
activos2 = activos.copy()
activos2["poblacion"]= np.log10(activos2["poblacion"])

In [14]:
pregunta2_histog = alt.Chart(activos2).mark_area().encode(
    x=alt.X('comuna',sort='-y', axis=alt.Axis(labels=False)),
    y=alt.Y(
            'poblacion:Q',
            axis=alt.Axis(title='poblacion (escala log10)')
           )
).properties(
    title='población por comuna',
    height=200,
    width=700)

#también agregados linea para la media,min y max de población
rule1 = alt.Chart(activos2).mark_rule(color='green').encode(
    y='mean(poblacion):Q',
    size=alt.value(2)
)
rule2 = alt.Chart(activos2).mark_rule(color='red').encode(
    y='max(poblacion):Q',
    size=alt.value(2)
)
rule3 = alt.Chart(activos2).mark_rule(color='yellow').encode(
    y='min(poblacion):Q',
    size=alt.value(2)
)


pregunta2_histog + rule1+rule2+rule3

alt.LayerChart(...)

**Comentarios:**

Note que no entregamos nombres de comunas ya que no sería entendible con la dimensión del gráfico (200x700). Se priorizó el tamaño de este por sobre los nombres de las comunas por lo que estamos bucando: comparar la variabilidad solamente. Si quisieramos diferenciar comunas especificamente podriamos hacer un gráfico más amplio (200x7000 por ejemplo) y buscar exhaustivamente las comunas que queremos, pero sería más eficiente usar el gráfico ya mostrado y buscar en el dataframe la información faltante. 

Por sobre otros gráficos:

* es lo mismo que un histograma y más limpio
* un gráfico de linea puede ser un poco vacío (literalmente), pero entrega la misma información. Se deja más claro cual es el minimo

Para responder a la pregunta necesitamos un indicador de cuan variable es la distribución. Para esto, notemos que desde la media, el mínimo y máximo se escapan en $\sim 2$ ordenes de magnitud en sus respectivas direcciones, sumado a que en este gráfico (escala logaritmica) desde el máximo vemos un descenso uniforme (aproximadamente al menos hasta un poco atrás del final de los datos) al mínimo, lo que se traduce en los datos reales como un descenso logaritmico. Entonces, en efecto, es altamente variable la población por comunas. 

Notemos que no usamos coefientes de dispersión ya que no tenemos punto de comparación.


## Ejercicio 4

(10 puntos)

`Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.`

**Respuesta:**

Para esto usaremos `confirmados`,`fallecidos`,`activos` y haremos gráficos de linea que funcionan bien asignar valor dado avance de fechas

In [15]:
df_pregunta4_confirmados = confirmados.groupby("fecha").sum().reset_index()

w=230
h=300

pregunta4_confirmados = alt.Chart(confirmados).transform_aggregate(
    groupby=['fecha'],
    suma = 'sum(casos_confirmados)',
).mark_line().encode(
    #note que tenemos muchas fechas asique mostramos por meses
    x=alt.X('fecha',axis=alt.Axis(title='fecha (meses)')),
    y=alt.Y('suma:Q', axis=alt.Axis(title = 'total casos confirmados'))
).properties(
    title='Total casos confirmados en el tiempo',
    width=w, 
    height=h
)

pregunta4_fallecidos =alt.Chart(fallecidos).transform_aggregate(
    groupby=['fecha'],
    suma = 'sum(fallecidos)',
).mark_line().encode(
    #note que tenemos muchas fechas asique mostramos por meses
    x=alt.X('fecha',axis=alt.Axis(title='fecha (meses)')),
    y=alt.Y('suma:Q', axis=alt.Axis(title = 'total fallecidos'))
).properties(
    title='Total cantidad fallecidos en el tiempo',
    width=w, 
    height=h
)


pregunta4_activos = alt.Chart(activos).transform_aggregate(
    groupby=['fecha'],
    suma = 'sum(casos_activos)',
).mark_line().encode(
    #note que tenemos muchas fechas asique mostramos por meses
    x=alt.X('fecha',axis=alt.Axis(title='fecha (meses)')),
    y=alt.Y('suma:Q', axis=alt.Axis(title = 'total casos activos'))
).properties(
    title='Total cantidad casos activos en el tiempo',
    width=w, 
    height=h
)

pregunta4_confirmados | pregunta4_fallecidos | pregunta4_activos

alt.HConcatChart(...)

**Comentarios:** Como se comentó en al respuesta de `pregunta 1, pregunta 3`, notamos en cada gráfico que a mitad de julio o poco antes, hay un cambio drástico en el comportamiento de la epidemia. De forma independiente

* las curvas de *cantidad de casos confirmados en el tiempo* y *Total cantidad fallecidos en el tiempo* tiene un comportamiento similar a la de la `pregunta 3`. Esto tiene sentido ya que son la suma de las curvas por grupo etareo que vimos y todas distribuian de forma muy similar. También podemos darnos cuenta de que a mitad de julio el crecimiento en las primeras 2 curvas dichas se estabiliza
* otra información que acompaña a la anterior es lo que podemos desprender del gráfico de total casos activos. Notamos que en Julio la cantidad de activos tiene un patron de decrecimiento mucho más estable que antes de la fecha. 

## Ejercicio 5

(10 puntos)

`Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.`

**Respuesta:**

Para esto usaremos `confirmados`. Como tenemos 16 regiones no es recomendable usar un gráfico de lineas como anteriormente, pero es mejor que otras opciones. Notemos que esto es debido a que graficaremos, dada una region y fecha, su media y alrededor una banda de error para no perder información de la distribución de cada region (información para calcular la media). 

In [16]:
band = alt.Chart(confirmados).mark_errorband(extent='ci').encode(
    x=alt.X('fecha:T',axis=alt.Axis(title='fecha (semanas)')),
    y=alt.Y('tasa:Q', axis=alt.Axis(title =  'media de tasa incidencia por region')),
    color = 'region'
).properties(
    title='Media tasa incidencia por region en el tiempo + error'
)


line =alt.Chart(confirmados).mark_line().transform_aggregate(
    groupby=['region','fecha'],
    mean = 'mean(tasa)'
).encode(
    #note que tenemos muchas fechas asique mostramos por meses
    x=alt.X('fecha:T',axis=alt.Axis(title='fecha (semanas)')),
    y=alt.Y('mean:Q', axis=alt.Axis(title = 'media de tasa incidencia por region')),
    color =alt.Color('region', scale=alt.Scale(scheme='category20')),
    tooltip=['region']
).properties(
    title='Media tasa incidencia por region en el tiempo',
    width=700, 
    height=600
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none')

selectors = alt.Chart(confirmados).mark_point().encode(
    x='fecha',
    opacity=alt.value(0),
).add_selection(
    nearest
)

points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

text1= line.mark_text(align='left', dx=-120, dy=-20,color='black').encode(
    text=alt.condition(nearest, 'region', alt.value(''))
)
text2 = line.mark_text(align='left', dx=100, dy=-20,color='black').encode(
    text=alt.condition(nearest, 'mean:Q', alt.value(''), format='.0f')
)

rules = alt.Chart(confirmados).mark_rule(color='gray').encode(
    x='fecha',
).transform_filter(
    nearest
)


line = alt.layer(
    line, selectors, points, rules, text2,text1
)


line | line+band

alt.HConcatChart(...)

**Comentarios:** 

nota: consideramos el segundo gráfico como un anexo solamente. Este nos sirve solo para detectar la dispersión que tienen las curvas, es muy denso para considerarlo como el principal.

Notemos que no es tan fácil detectar regiones por la cantidad de variables.

En el primer gráfico, Tarapaca es el que parece tener máxima tasa, pero esto es solo el promedio, lo que se concreta en el segundo gráfico donde la variabilidad de la curva de Arica y Parinacota hace que supere en gran medida la antes dicha.

Sobre las distribuciones, podemos notar que las regiones de Tarapaca, Metropolitana, Antofagasta, Arica y Magallanes tienen mayores indices. Entre estas, respecto a Magallanes, vemos que lo hecho en la primera tarea del curso toma forma gráfica, ya que la curva correspondiente tiene una pendiente no amortiguada en las fechas más actuales (lo que hace notar la preocupación hace semanas atrás), como son las de las otras regiones en esta clasificación (índice alto). Por otro lado, todas las otras regiones tienen un comportamiento similar, creciente pero más controlado, a excepción de Los lagos, Araucania y Los Rios.

Por último, relacionado con lo comentado en preguntas anteriores, vemos que a mitad de julio hay un cambio positivo en la mayoria de curvas con mayores indices y las con menor aproximadamente se mantienen.

## Ejercicio 6

(10 puntos)

`¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.`

**Respuesta:**

Para esto usaremos `confirmados` que posee lo que necesitamos. Realizaremos un _scatter plot_ de cada fecha por separado para y veremos si hay alguna diferencia. Además

Veamos como es el gráfico incidencia vs casos confirmados:

In [17]:
df_pregunta6 = confirmados.copy()

w=700
h=200

pregunta4_1 = alt.Chart(df_pregunta6[df_pregunta6["fecha"]== '2020-04-13']).mark_circle().encode(
    x=alt.X('casos_confirmados'
            , axis = alt.Axis(title='casos confirmados')),
    y=alt.Y('tasa', axis = alt.Axis(title='tasa incidencia')),
    color=alt.Color('region', scale=alt.Scale(scheme='category20')),
    tooltip=['region'],
     size=alt.Size('poblacion', scale=alt.Scale(range=[0, 500]))
).properties(
    title='tasa de incidencia respecto a casos confirmados, 2020-04-13',
    width=w, 
    height=h
)

pregunta4_2 = alt.Chart(df_pregunta6[df_pregunta6["fecha"]== '2020-11-6']).mark_circle().encode(
    x=alt.X('casos_confirmados'
            , axis = alt.Axis(title='casos confirmados')),
    y=alt.Y('tasa', axis = alt.Axis(title='tasa incidencia')),
    color=alt.Color('region', scale=alt.Scale(scheme='category20')),
    tooltip=['region'],
     size=alt.Size('poblacion', scale=alt.Scale(range=[0, 500]))
).properties(
    title='tasa de incidencia respecto a casos confirmados, 2020-11-6',
    width=w, 
    height=h
)

alt.vconcat(pregunta4_1, pregunta4_2)

alt.VConcatChart(...)

**Comentarios:** 

Se considero relevante usar tamaño para los puntos respecto a la población ya que es una información que está en la tasa de forma directa

$$tasa = 100000 \frac{confirmados}{poblacion}$$

, pero más relevante es que podemos notar que las comunas con mayor población se ubican, mayoritariamente con mayor casos confirmados (lo que es razonable), esto más marcado en el segundo gráfico. Ahora, si comparamos los dos gráficos, podemos darnos cuenta de que son "complementarios" en el sentido de que podriamos trazar una recta que separa estos dos conjuntos con poco error respecto a la población total separada (considerar los elementos con peso la población). Dicho de otra forma, del 3 de abril al 6 de noviembre la relación $\frac{tasa}{casos confirmados}$ aumentó, es decir, en los ajustes lineales a cada distribucion de puntos , el segundo gráfico tiene mayor pendiente. 

Respecto comentar sobre las regiones (alguna en específico) no podemos decir mucho más que las que tiene altos indices y altos casos confirmados (por ejemplo, Santiago y Valparaiso), no se notan muchas diferencias.

## Ejercicio 7

(10 puntos)

`1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico.`

`2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.`

`Entrega los pros y contras de cada uno de estos enfoques.`

**Respuesta:**

Para esto usaremos `activos`

In [18]:
alt.Chart(activos).mark_line().encode(
    #note que tenemos muchas fechas asique mostramos por semanas
    x=alt.X('fecha',axis=alt.Axis(title='fecha (semanas)')),
    y=alt.Y('casos_activos:Q', axis=alt.Axis(title = 'casos activos')),
    color =alt.Color('comuna'),
    tooltip=['comuna','casos_activos']
).properties(
    title='Evolución de casos activos',
    width=700, 
    height=400
)

alt.Chart(...)

primero veamos cual es la maxima cantidad de comunas por region para saber cuan entendible será un gráfico de lineas

In [19]:
activos.groupby(["region","comuna"]).agg(
        empty=("codigo_region", lambda x: x.sum()/x.sum()),
    ).reset_index().groupby("region").sum().max()[0]

52

tampoco es una buena idea. Veamos como son los gráficos

In [20]:
pregunta7_base = alt.Chart(activos).mark_line().encode(
    x='fecha:T',
    y=alt.Y("casos_activos",axis = alt.Axis(title = 'casos activos')),
    color='comuna',
    tooltip=['comuna']
).properties(
    title='Evolución de casos activos de comunas por region',
    width=200,
    height=200
).facet(
        title='Evolución de casos activos de comunas por región',
    facet='region:N',
    columns=4
)

pregunta7_base

alt.FacetChart(...)

**Comentarios:** 

Para fácil lectura, se entregan las caracteristicas en un listado

Sobre la gráfica 1

* contra gráfico 1: La carac. más fuerte que tiene en contra el primer gráfico es que no se entiende que cómo caracterizar las curvas exceptuando algún rango donde alcanza un peak y es perteptible. Por ejemplo, es dificil afirmar algo si bajamos de 600 en el eje Y. 
* contra gráfico 1: no disponemos del listado de las comunas de manera fácil. Podemos buscar con el cursor y haciendo zoom ya que usamos `.interactive()`, pero esto sería de manera exhaustiva.
* contra gráfico 1: acompañando a lo anterior, como tenemos mas de 300 elementos, es inhumano percibir una gama de más de 300 colores, con lo que solo podría dar información de algunos tramos extremos como se dijo.
* pro gráfica 1: podemos darnos cuenta facilmente, a excepción de un elemento (Punta Arenas), que las comunas, en conjunto, alcanzaron una zona peak en una vecindad de la mitad de julio, antes y después de esto el crecimiento/decrecimiento es más amortiguado, incluso se notan distribuciones similares.

Sobre la gráfica 2

* pro gráfica 2: a diferencia de la anterior, en la mayoria de regiones sí disponemos un número de comunas que hace más reconocible cuando los asociamos con colores. Casos en que no ocurre son en región Metropolitana, Bio Bio donde hay valores muy bajos,etc. Aún así no es muy informativo.
* contra gráfica 2: No podemos comparar entre todas las regiones. Una solución podría ser hacer un gráfico tipo [ridgeline](https://altair-viz.github.io/gallery/ridgeline_plot.html) o disponerlos horizontalmente. Note que esto es mejor opción que `gráfico 1` ya que tendremos información por regiones, pero tiene una extensión larga en una pantalla normal.
* contra gráfico 2: Como el máximo qlobal alcanzado en la region metropolitana  es muy alto en comparación con el mínimo o la media, no es fácil caracterizar las curvas que están por debajo de la media. Puede ser solución reescalar los otros gráficos pero perdemos información en cómo se compara Santiago con las de su fila.
* pro gráfica 2: podemos encontrar facilmente la comuna(s) que tiene(n) máximo(s) por región y así detectar zonas críticas localmente.

En cualquier caso, quiza es mejor caracterizar las regiones/comunas a traves de otros indicadores/estadisticas que resuman de mejor manera la información y que su representación gráfica sea más esclarecedora. Esto perseguido con la idea de por qué queremos estas gráficas. Como se dijo, si queremos buscar regiones críticas, se cumple la función, pero no se alcanza mucha mas rigurosidad. Otra opción sería hacer un heatmpap, pero se sale del segundo punto solicitado.

In [21]:
# descomentar para ver idea

# alt.Chart(activos).mark_rect().encode(
#     y='fecha',
#     x='comuna:N',
#     color=alt.Color('casos_activos:Q',scale=alt.Scale(scheme='viridis'))
# )

## Ejercicio 8

(10 puntos)

`
Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.`


Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [22]:
region_names = {
    "Araucania": "La Araucanía",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    
#     "Biobio" : "Biobío",
#     "Del Libertador General Bernardo O’Higgins" : "Libertador General Bernardo O’Higgins",
#     "Tarapacá": "Tarapaca",
#     "Nuble": "Ñuble"
}

**Respuesta :**

Buscamos un gráfico de barras con eje X las regiones, por cada region 3 elementos que serán los dichos y el eje Y el valor asociado después de que normalicemos, esto dado que solo queremos comparar entre regiones y los máximos de `confirmados` son mucho mayores a los otros. Usaremos `activos`,`fallecidos`,`confirmados`. Primero calculemos por regiones y después los juntamos en un dataframe
* media de casos activos
* máximo de casos confirmados
* máximo de fallecidos
y cambiemos los nombres de las regiones para estandarizar

In [23]:
pregunta8_activos= activos.groupby(
    "region").mean().rename(index = region_names).reset_index().loc[:,["region","casos_activos"]]

pregunta8_max_confirmados = confirmados.groupby(
    "region").max().rename(index = region_names).reset_index().loc[:,["region","casos_confirmados"]]

pregunta8_max_fallecidos = fallecidos.groupby(
    "region").max().rename(index = region_names).reset_index().loc[:,["region","fallecidos"]]

aux = pd.merge(pregunta8_activos, pregunta8_max_confirmados, on=['region'])
aux.head(20)

,region,casos_activos,casos_confirmados
0,Antofagasta,78.112795,12831.0
1,Arica y Parinacota,100.310606,10530.0
2,Atacama,33.713805,4820.0
3,Aysén del General Carlos Ibáñez del Campo,4.406061,1086.0
4,Coquimbo,33.002020,5099.0
5,Los Lagos,24.295455,7812.0
6,Maule,26.694949,5141.0
7,Metropolitana de Santiago,159.546620,28266.0


Faltan datos asique busquemos cómo están escritas las regiones y eliminemos de forma exhaustiva los que necesitamos estandarizar

In [24]:
set1=set(pregunta8_activos["region"])
set2=set(pregunta8_max_confirmados["region"])
set3=set(pregunta8_max_fallecidos["region"])

set.union(set1, set2,set3)

{'Antofagasta',
 'Araucanía',
 'Arica y Parinacota',
 'Atacama',
 'Aysén',
 'Aysén del General Carlos Ibáñez del Campo',
 'Biobio',
 'Biobío',
 'Coquimbo',
 'Del Libertador General Bernardo O’Higgins',
 'La Araucania',
 'La Araucanía',
 "Libertador General Bernardo O'Higgins",
 'Los Lagos',
 'Los Rios',
 'Los Ríos',
 'Magallanes y de la Antártica Chilena',
 'Magallanes y la Antartica',
 'Maule',
 'Metropolitana de Santiago',
 'Nuble',
 'Tarapaca',
 'Tarapacá',
 'Total',
 'Valparaiso',
 'Valparaíso',
 'Ñuble'}

In [25]:
region_names = {
# los elementos que ya cambiamos 
    
#     "Araucania": "La Araucanía",
#     "Aysen": "Aysén del General Carlos Ibáñez del Campo",
#     "Magallanes": "Magallanes y de la Antártica Chilena",
#     "Metropolitana": "Metropolitana de Santiago",
#     "O’Higgins": "Libertador General Bernardo O'Higgins",
    'Biobio':'Biobío',    
    "La Araucania":"La Araucanía",
    'Araucanía':"La Araucanía",
    'Aysén':'Aysén del General Carlos Ibáñez del Campo',
    'Aysen':'Aysén del General Carlos Ibáñez del Campo',
    'Los Rios':'Los Ríos',
    'Magallanes y la Antartica':'Magallanes y de la Antártica Chilena',
    'Magallanes':'Magallanes y de la Antártica Chilena',
    'Tarapaca':'Tarapacá',
    'Valparaiso':'Valparaíso', 
    'Metropolitana':'Metropolitana de Santiago',
    'Nuble':'Ñuble',
    
    'Del Libertador General Bernardo O’Higgins':"Libertador General Bernardo O'Higgins",
    'O’Higgins':"Libertador General Bernardo O'Higgins"
}
pregunta8_activos= activos.groupby(
    "region").mean().rename(index = region_names).reset_index().loc[:,["region","casos_activos"]]

pregunta8_max_confirmados = confirmados.groupby(
    "region").max().rename(index = region_names).reset_index().loc[:,["region","casos_confirmados"]]

pregunta8_max_fallecidos = fallecidos.groupby(
    "region").max().rename(index = region_names).reset_index().loc[:,["region","fallecidos"]]

df_pregunta8 = pd.merge(pregunta8_activos, pregunta8_max_confirmados, on=['region'])
df_pregunta8 =pd.merge(df_pregunta8,pregunta8_max_fallecidos,on =["region"] )

#tenemos que normalizar cada columna (posterior)

# for column in ["casos_activos","casos_confirmados","fallecidos"]:
#     df_pregunta8[column] = df_pregunta8[column]/(df_pregunta8[column].max())

df_pregunta8

,region,casos_activos,casos_confirmados,fallecidos
0,Antofagasta,78.112795,12831.0,562.0
1,Arica y Parinacota,100.310606,10530.0,207.0
2,Atacama,33.713805,4820.0,106.0
3,Aysén del General Carlos Ibáñez del Campo,4.406061,1086.0,14.0
4,Biobío,46.290174,6401.0,610.0
5,Coquimbo,33.002020,5099.0,270.0
6,Libertador General Bernardo O'Higgins,24.648760,9046.0,544.0
7,La Araucanía,16.421402,5836.0,229.0
8,Los Lagos,24.295455,7812.0,203.0
9,Los Ríos,17.915404,2940.0,55.0


Con lo cual ya obtuvimos las 16 regiones que buscabamos. Ahora despivotamos las columnas

In [26]:
df_pregunta8_2 = pd.concat(
    [df_pregunta8.melt(
    id_vars=['region']
    ,value_vars="casos_activos"
    ,var_name='clasificacion'
    , value_name='valor'),
    df_pregunta8.melt(
    id_vars=['region']
    ,value_vars="casos_confirmados"
    ,var_name='clasificacion'
    , value_name='valor'),
    df_pregunta8.melt(
    id_vars=['region']
    ,value_vars="fallecidos"
    ,var_name='clasificacion'
    , value_name='valor')])
df_pregunta8_2['valor']=np.log10(df_pregunta8_2['valor'])

df_pregunta8_copy=df_pregunta8.copy()

for column in ["casos_activos","casos_confirmados","fallecidos"]:
    df_pregunta8_copy[column] = df_pregunta8_copy[column]/(df_pregunta8_copy[column].max())

df_pregunta8_22 = pd.concat(
    [df_pregunta8_copy.melt(
    id_vars=['region']
    ,value_vars="casos_activos"
    ,var_name='clasificacion'
    , value_name='valor'),
    df_pregunta8_copy.melt(
    id_vars=['region']
    ,value_vars="fallecidos"
    ,var_name='clasificacion'
    , value_name='valor'),
    df_pregunta8_copy.melt(
    id_vars=['region']
    ,value_vars="casos_confirmados"
    ,var_name='clasificacion'
    , value_name='valor')])

Veamos como es el gráfico buscado

In [27]:
pregunta8_bar = alt.Chart(df_pregunta8_2).mark_bar().encode(
    x=alt.X('clasificacion:N', title=None, axis=alt.Axis(labels=False)),
    y=alt.Y('valor',sort='-y'),
    color=alt.Color('clasificacion:N'),
    column=alt.Column('region:O', title="Máximo fallecidos, máximo casos confirmados y promedio casos activos (escala log10)"
                      , header=alt.Header(labelAngle=7))
).properties(
    width=20
).configure_facet(
    spacing=40
    )

#para el otro gráfico tenemos

pregunta8_bar2 = alt.Chart(df_pregunta8_22).mark_bar().encode(
    x=alt.X('clasificacion:N', title=None, axis=alt.Axis(labels=False)),
    y=alt.Y('valor'),
    color=alt.Color('clasificacion:N'),
    column=alt.Column('region:O', title="Máximo fallecidos, máximo casos confirmados y promedio casos activos (normalizados)"
                      , header=alt.Header(labelAngle=7))
).properties(
    width=20
).configure_facet(
    spacing=40
    )

display(pregunta8_bar)
display(pregunta8_bar2)

alt.Chart(...)

alt.Chart(...)

**Comentarios:** 

Nota: tenga en cuenta que para encontrar la región de cada bloque con barras debe encontrar el nombre que termina en el tope de este bloque.

Podriamos considerar estos dos gráficos. En el primero ganamos que sabemos más información de los datos más pequeños y en el segundo vemos cual es la region que tiene más grandes indicadores, en este caso Santiago es mayor en todo. En el segundo podemos comparar porcentualmente de forma fácil. Por ejemplo, en todos los elementos de `clasificacion` encontramos que las regiones tienen bajo 50% a excepción de casos activos de Arica. También, los fallecidos entre regiones sin Valparaiso, comparados con los de Santiago, no superan el 10% . Similar para con `casos confirmados` con un 50% y `casos activos` y un 60%. Por otro lado, en el primer gráfico poemos caracterizar las diferencias por orden de magnitud. 

En caso de escoger un gráfico tomaría el primero ya que se pide mantener valores reales, pero hacer un gráfico de este estilo con valores reales conduce a perder los datos de `casos activos` (muy pequeños), entonces es lo más cercano, en este gráfico, a no perder información y ser claro. Estos dos gráficos complementan bien la información buscada.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [28]:
import geopandas as gpd
from pathlib import Path

In [29]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.columns = map(str.lower, regiones.columns)
regiones.head()

,region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [30]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

**Respuesta:**


Recordemos la estructura de los dataframe

In [ ]:
k=1
display(confirmados.head(k))
display(activos.head(k))
display(fallecidos.head(k))
display(regiones.head(k))

Necesitamos `codigo_region` asique estandarizamos el nombre de columna para todos los dataframe:

In [ ]:
confirmados=confirmados.rename(columns= {'region_id':'codigo_region'})
regiones=regiones.rename(columns= {'codregion':'codigo_region'})

notemos que no necesitamos la información por comunas, entonces debemos agrupar y transformar algunos valores tal que nos de información de las regiones, en concreto, la tasa, casos confirmados y casos activos

In [ ]:
# buscamos casos confirmados y tasa por región:

pregunta9_confirmados = pd.merge(
    confirmados.groupby(['region','fecha']).sum().reset_index().loc[:,['casos_confirmados','fecha','region']]
    ,confirmados.groupby(['region','codigo_region','fecha']).sum().reset_index().loc[:,['region','codigo_region','poblacion']]
    ,how='inner',on=['region']
    ).assign(
       tasa=lambda df: df['casos_confirmados']*100000/df['poblacion']
).drop_duplicates().reset_index()
pregunta9_confirmados

seleccionemos las columnas que nos interesan de cada uno: eliminamos las columnas que ya están en al anterior (orden anterior), mantenemos las faltantes y las que hacen en nexo para un merge. Además agrupamos en caso de ser necesario

In [ ]:
pregunta9_activos=activos.groupby(['region','fecha']).agg(casos_activos=("casos_activos", "sum")).reset_index().loc[:,['region','fecha','casos_activos']]
pregunta9_fallecidos=fallecidos.loc[:,['region','fecha','fallecidos']]
regiones= regiones.loc[:,['region','geometry']]

display(activos.groupby(['region','fecha']).agg(casos_activos=("casos_activos", "sum")).reset_index().loc[:,['region','fecha','casos_activos']].head(2))
print(activos.groupby(['region','fecha']).agg(casos_activos=("casos_activos", "sum")).reset_index().shape)

display(fallecidos.loc[:,['region','fecha','fallecidos']].head(2))
print(fallecidos.shape)

display(regiones.loc[:,['region','geometry']].head(2))
print(regiones.shape)

recordemos que queremos algo como los mismos nombres de `df_pregunta8_2`

In [ ]:
set(df_pregunta8_2['region'])

In [ ]:
set1=set(confirmados['region'])
set2=set(activos['region'])
set3=set(fallecidos['region'])
set4=set(regiones['region'])

#eliminamos los terminos que sabemos que están bien respecto a lo que buscamos
set.union(set1,set2,set3,set4)-set(df_pregunta8_2['region'])

In [ ]:
region_names_pregunta9 = {
    "Araucania": "La Araucanía",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    
# COMO DEBERIAN SER    
    
#  {'Antofagasta',
#  'Arica y Parinacota',
#  'Atacama',
#  'Aysén del General Carlos Ibáñez del Campo',
#  'Biobío',
#  'Coquimbo',
#  'La Araucanía',
#  "Libertador General Bernardo O'Higgins",
#  'Los Lagos',
#  'Los Ríos',
#  'Magallanes y de la Antártica Chilena',
#  'Maule',
#  'Metropolitana de Santiago',
#  'Tarapacá',
#  'Valparaíso',
#  'Ñuble'}   
    
    'Araucanía': "La Araucanía",
     'Aysen':"Aysén del General Carlos Ibáñez del Campo",
     'Aysén':"Aysén del General Carlos Ibáñez del Campo",
     'Biobio': 'Biobío',
     'Del Libertador General Bernardo O’Higgins':"Libertador General Bernardo O'Higgins",
     'La Araucania': "La Araucanía",
     'Los Rios': 'Los Ríos',
     'Magallanes':'Magallanes y de la Antártica Chilena',
     'Magallanes y la Antartica':'Magallanes y de la Antártica Chilena',
     'Metropolitana':'Metropolitana de Santiago',
     'Nuble':'Ñuble',
     'O’Higgins':"Libertador General Bernardo O'Higgins",
     'Región Metropolitana de Santiago':'Metropolitana de Santiago',
     'Región de Antofagasta':'Antofagasta',
     'Región de Arica y Parinacota':'Arica y Parinacota',
    
     'Región de Atacama':'Atacama',
     'Región de Aysén del Gral.Ibañez del Campo':"Aysén del General Carlos Ibáñez del Campo",
     'Región de Coquimbo':'Coquimbo',
     'Región de La Araucanía': "La Araucanía",
     'Región de Los Lagos':'Los Lagos',
     'Región de Los Ríos':'Los Ríos',
     'Región de Magallanes y Antártica Chilena':'Magallanes y de la Antártica Chilena',
     'Región de Tarapacá':'Tarapacá',
     'Región de Valparaíso':'Valparaíso',
     'Región de Ñuble':'Ñuble',
     'Región del Bío-Bío':'Biobío',
     "Región del Libertador Bernardo O'Higgins":"Libertador General Bernardo O'Higgins",
     'Región del Maule':'Maule',
     'Tarapaca':'Tarapacá',
     'Valparaiso':'Valparaíso'
}

In [ ]:
regiones['region']=regiones['region'].replace(region_names_pregunta9)
pregunta9_confirmados['region']=pregunta9_confirmados['region'].replace(region_names_pregunta9)
pregunta9_activos['region']=pregunta9_activos['region'].replace(region_names_pregunta9)
pregunta9_fallecidos['region']= pregunta9_fallecidos['region'].replace(region_names_pregunta9)

In [ ]:
casos_geo = pd.merge(pregunta9_confirmados,pregunta9_activos, how='inner', on=['region','fecha'])
casos_geo = pd.merge(casos_geo,pregunta9_fallecidos, how='inner', on=['region','fecha'])
casos_geo = pd.merge(casos_geo,regiones, how='inner', on=['region'])
display(casos_geo.head())
print(casos_geo.shape)

obtuvimos el mismo número máximo de filas asique no perdimos ni agregamos datos.

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
casos_geo.head(2)

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo[
                    casos_geo["fecha"]==pd.to_datetime(fecha)
                    ]
    
    chart = alt.Chart(data).mark_geoshape().encode(
       color = col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = widgets.Dropdown(
    options=['poblacion', 'casos_confirmados', 'tasa','casos_activos','fallecidos'],
    description='columna',
    continuous_update=False
)

In [ ]:
fecha_widget = widgets.DatePicker(
    description='fecha',
    continuous_update=False
)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** RESPONDE AQUÍ